In [1]:
import os

from utils import common, model
from victim.blackbox import Blackbox
from datasets import get_dataset
import numpy as np
import torch

import torchattacks

device = common.device_dealer(3)
blackbox = Blackbox.from_modeldir('/mnt/ywb/results/victim/gtsrb-blackbox', device)
surrogate = model.Model('/mnt/ywb/results/adversary/gtsrb-imagenet-balanced', 'Classifier', 32, 'GTSRB', 100, device,
                        complexity=64, batch_size=128,
                        resume='/mnt/ywb/results/adversary/gtsrb-imagenet-balanced/checkpoint.1000.pth.tar', train_criterion='SCE')
imagenet = get_dataset('ImageNet64', 32, True)
indexes = np.load(os.path.join(surrogate.model_dir, 'selected_.npy'))

=> loading checkpoint '/mnt/ywb/results/victim/gtsrb-blackbox/checkpoint.pth.tar'
=> loaded checkpoint (epoch 56, acc=97.81)
=> done loading GTSRB (test) with 12630 examples
=> loading checkpoint '/mnt/ywb/results/adversary/gtsrb-imagenet-balanced/checkpoint.1000.pth.tar'
=> loaded checkpoint '/mnt/ywb/results/adversary/gtsrb-imagenet-balanced/checkpoint.1000.pth.tar' (epoch 96)


In [3]:
imagenet[0]

(tensor([[[0.1922, 0.2431, 0.1804,  ..., 0.3059, 0.2706, 0.3255],
          [0.2196, 0.2588, 0.1922,  ..., 0.2902, 0.2784, 0.3176],
          [0.2667, 0.2863, 0.2314,  ..., 0.2863, 0.2471, 0.2745],
          ...,
          [0.7176, 0.7176, 0.7176,  ..., 0.8039, 0.7882, 0.7725],
          [0.7255, 0.7294, 0.7137,  ..., 0.8039, 0.7843, 0.7725],
          [0.7137, 0.7255, 0.7137,  ..., 0.8039, 0.7922, 0.7804]],
 
         [[0.2078, 0.2667, 0.2078,  ..., 0.3176, 0.2824, 0.3451],
          [0.2353, 0.2824, 0.2235,  ..., 0.2980, 0.2863, 0.3373],
          [0.2784, 0.3059, 0.2588,  ..., 0.2902, 0.2549, 0.2863],
          ...,
          [0.6902, 0.6980, 0.6941,  ..., 0.7725, 0.7569, 0.7451],
          [0.7020, 0.7059, 0.6863,  ..., 0.7686, 0.7569, 0.7451],
          [0.6902, 0.7020, 0.6902,  ..., 0.7686, 0.7608, 0.7451]],
 
         [[0.1961, 0.2549, 0.1843,  ..., 0.3176, 0.2471, 0.2902],
          [0.2235, 0.2706, 0.2000,  ..., 0.2980, 0.2471, 0.2784],
          [0.2627, 0.2941, 0.2471,  ...,

In [4]:
query_instance = common.QueryWrapper(imagenet, indexes)
results = common.query(blackbox, query_instance, device=device)
train_instance = common.QuerySubset(imagenet, indexes, results)

ModuleNotFoundError: No module named 'pydev_jupyter_utils'

In [ ]:
sm = np.load(os.path.join(surrogate.model_dir, 'statistic_matrix_.npy'))

In [ ]:
surrogate.train(train_instance, 'ModelClass')

In [ ]:
model.train_model(surrogate.model, train_instance, surrogate.model_dir, 128, testset=surrogate.testset, device=device, criterion_train=model.soft_cross_entropy, optimizer=surrogate.optimizer)

In [ ]:
selecting_pool = set(range(len(imagenet))).difference_update(indexes)
new_indices =  np.random.choice(list(selecting_pool), 10000, replace=False)
query_instance = common.QueryWrapper(imagenet, new_indices)
results = common.query(blackbox, query_instance, device=device)
train_instance.extend(new_indices, results)

In [ ]:
import torchattacks
import numpy as np
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm
attacker = torchattacks.CW(surrogate.model, c=0.1, steps=200)
attacker.set_attack_mode('targeted')
index_list = list(range(len(imagenet)))
random_selection = np.random.choice(index_list, 1000, replace=False)
random_imagenet = common.Subset(imagenet, random_selection)

all = 0
epsilon=0.
for x_t, y_t in DataLoader(random_imagenet, 256, False):
    target = torch.ones(x_t.size(0), dtype=int)
    x_adv = attacker(x_t, target)
    epsilon += (x_adv.cpu() - x_t).reshape(x_t.size(0), 32*32*3).norm(dim=1).sum()
    y_adv = surrogate(x_adv.to(device)).argmax(1)
    all += (target == y_adv.cpu()).sum()

In [ ]:
import torch
import numpy as np
x = np.load('/mnt/ywb/results/adversary/gtsrb-imagenet-targeted/statistic_matrix_.npy')